<a href="https://colab.research.google.com/github/Le4o/topicos-avancados-ic/blob/main/S%C3%A9ries_Temporais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Séries Temporais
#### Exercício da matéria de Tópicos Avançados em Inteligência Computacional

Grupo:
- João Victor de Sledz Bulhões
- Leonardo de Andrade Santana
- Lis da Silva Azevedo

### Definição do Exercício

A atividade é **prever para as próximas 6h o valor da concentração de PM2.5**.
Utilize os conceitos aprendidos até aqui, **testando diferentes modelos de IA** e **diferentes abordagens para conseguir prever com o melhor desempenho o valor das próximas 6h da concentração de PM2.5 para uma série histórica de sua escolha.**  
1.   Escolha uma estação de monitoramento de uma das cidades com que irá trabalhar; 
2.   Faça uma análise exploratória dos dados, avaliando estatisticamente e graficamente como os dados se comportam;
3.   Explore as técnicas **MLP, RNN simples, LSTM, GRU, CNN1D, FCN+LSTM e Conv2DLSTM**, utilizando a camada **TimeDistributed**;
4.   Escolha, com base na avaliação do **loss** e das métricas **MAE, MSE, NMSE, r, R2 e Fac2** qual foi o melhor modelo (e apresente justificando a resposta);
5.   Use o ano de 2015 somente para fins de teste.

### Observação: 
*   *Apresente os resultados de forma lógica, organizada e que seja reprodutível pelo professor (linhas comentadas);*
*   *Apresentação de métricas, gráficos, são essenciais para entendimento dos modelos e justificativas.*

### Opcionais:

*   Pode-se utilizar as informações meteorológicas para melhorar a qualidade do modelo final;
*   Interessante, mas não obrigatório, a utilização de wavelets para feature augmentation.


### Links:

* [Código para o colab (referência)](https://colab.research.google.com/drive/1b5177FoO4JsyOKN8dmbmTjvPi32b95of?usp=sharing) 

* [Link da base de dados da atividade](hhttps://archive.ics.uci.edu/ml/datasets/PM2.5+Data+of+Five+Chinese+Cities)

* [Link para download da base de dados da atividade](https://drive.google.com/file/d/19FsUZXhakjteNoC3nsqUe2aTq3s7GoQI/view?usp=drive_web&authuser=1)
